<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [29]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

In [30]:
df_list = []
for url in archivos_anio:
    df_temp = pd.read_csv(url)
    df_temp.columns = [col.lower() for col in df_temp.columns]
    df_list.append(df_temp)

df_anio = pd.concat(df_list, ignore_index=True)
df_anio.head()

,codigo_iso,anio,indice,ranking
0,AFG,2001,35.5,59.0
1,AGO,2001,30.2,50.0
2,ALB,2001,NaN,NaN
3,AND,2001,NaN,NaN
4,ARE,2001,NaN,NaN


**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

In [31]:
print('Antes')
print(df_codigos['codigo_iso'].value_counts().head())

Antes
codigo_iso
ZWE    2
AFG    1
ALB    1
AGO    1
ARE    1
Name: count, dtype: int64


In [32]:
print(df_codigos[df_codigos['codigo_iso'] == 'ZWE'])
df_codigos = df_codigos[df_codigos['pais'] != 'malo']

    codigo_iso      pais
179        ZWE  Zimbabue
180        ZWE      malo


In [33]:
print('\nDespués')
print(df_codigos['codigo_iso'].value_counts().head())


Después
codigo_iso
AFG    1
AGO    1
ALB    1
AND    1
ARE    1
Name: count, dtype: int64


**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.

In [38]:
df = pd.merge(df_anio, df_codigos, on='codigo_iso', how='inner')
df.head()

,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.5,59.0,Afghanistán
1,AGO,2001,30.2,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?



    

In [40]:
print(f"Cantidad de filas: {df.shape[0]}")

Cantidad de filas: 3060


* ¿Cuántas **columnas** tiene el DataFrame?


In [41]:
print(f"Cantidad de columnas: {df.shape[1]}")

Cantidad de columnas: 5


* ¿Cuáles son los **nombres de las columnas**?

In [42]:
print(df.columns)

Index(['codigo_iso', 'anio', 'indice', 'ranking', 'pais'], dtype='object')


* ¿Qué **tipo de datos** tiene cada columna?

In [45]:
df.dtypes

,0
codigo_iso,object
anio,int64
indice,float64
ranking,float64
pais,object


Las columnas codigo_iso y pais son del tipo string. La columna de anio es de tipo entero, y las de indice y ranking son de tipo flotante

* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

In [48]:
print("¿codigo_iso solo tiene datos del tipo object?:", df["codigo_iso"].dtype == "object")
print("¿anio solo tiene datos del tipo entero?:", df["anio"].dtype == "int64")
print("¿indice solo tiene datos del tipo flotante?:", df["indice"].dtype == "float64")
print("¿ranking solo tiene datos del tipo flotante?:", df["ranking"].dtype == "float64")
print("¿pais solo tiene datos del tipo object?:", df["pais"].dtype == "object")

¿codigo_iso solo tiene datos del tipo object?: True
¿anio solo tiene datos del tipo entero?: True
¿indice solo tiene datos del tipo flotante?: True
¿ranking solo tiene datos del tipo flotante?: True
¿pais solo tiene datos del tipo object?: True


#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.

In [49]:
display(df.describe())

,anio,indice,ranking
count,3060.000000,2664.000000,2837.000000
mean,2009.941176,205.782316,477.930913
std,5.786024,2695.525264,6474.935347
min,2001.000000,0.000000,1.000000
25%,2005.000000,15.295000,34.000000
50%,2009.000000,28.000000,70.000000
75%,2015.000000,41.227500,110.000000
max,2019.000000,64536.000000,121056.000000


  ¿Qué observas sobre los valores de `indice` y `ranking`?

Los valores máximos y mínimos de indice y ranking son extremos respecto al sus respectivos promedios.

* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?

In [50]:
print(f"Indice minimo: {df['indice'].min()}")
print(f"Indice maximo: {df['indice'].max()}")
print(f"Indice promedio: {df['indice'].mean()}")

Indice minimo: 0.0
Indice maximo: 64536.0
Indice promedio: 205.7823160660661


* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

In [53]:
print("Pais con indice minimo (mayor libertad):")
display(df.loc[df['indice'].idxmin()])

print("\nPais con indice maximo (menor libertad):")
display(df.loc[df['indice'].idxmax()])

print("\nPais con mejor ranking (el mas bajo):")
display(df.loc[df['ranking'].idxmin()])

print("\nPais con peor ranking (el mas alto):")
display(df.loc[df['ranking'].idxmax()])

Pais con indice minimo (mayor libertad):


,1304
codigo_iso,DNK
anio,2008
indice,0.0
ranking,2.0
pais,Dinamarca



Pais con indice maximo (menor libertad):


,2069
codigo_iso,KSV
anio,2014
indice,64536.0
ranking,120614.0
pais,Kosovo



Pais con mejor ranking (el mas bajo):


,53
codigo_iso,FIN
anio,2001
indice,0.5
ranking,1.0
pais,Finlandia



Pais con peor ranking (el mas alto):


,2249
codigo_iso,KSV
anio,2015
indice,64527.0
ranking,121056.0
pais,Kosovo


#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?

In [55]:
print("Cantidad de valores nulos en cada columna:")
display(df.isnull().sum())

Cantidad de valores nulos en cada columna:


,0
codigo_iso,0
anio,0
indice,396
ranking,223
pais,0


* ¿Qué proporción de observaciones tienen valores faltantes?

In [57]:
print("Proporcion de observaciones de los valores faltantes:")
display(df.isnull().sum() / len(df) * 100)

Proporcion de observaciones de los valores faltantes:


,0
codigo_iso,0.000000
anio,0.000000
indice,12.941176
ranking,7.287582
pais,0.000000


* ¿Hay columnas con más del 30% de datos faltantes?

In [60]:
datos_faltantes = df.isnull().sum() / len(df) * 100
columnas_muchos_datos_faltantes = datos_faltantes[datos_faltantes > 30]

if not columnas_muchos_datos_faltantes.empty:
    print("Las columnas con mas del 30% de datos faltantes son:")
    display(columnas_muchos_datos_faltantes)
else:
    print("No hay columnas con mas del 30% de datos faltantes")

No hay columnas con mas del 30% de datos faltantes


#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?

In [61]:
print(f"Cantidad de paises distintos: {df['pais'].nunique()}")

Cantidad de paises distintos: 179


* ¿Cuántos años distintos (`anio`) hay representados?

In [62]:
print(f"Cantidad de años distintos: {df['anio'].nunique()}")

Cantidad de años distintos: 17


* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

In [71]:
filas_duplicadas = df.duplicated().sum()
print(f"Cantidad de filas duplicadas: {filas_duplicadas}")

Cantidad de filas duplicadas: 0


#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.

In [70]:
# Check for codigo_iso with more than one country name
conteo_paises = df.groupby('codigo_iso')['pais'].nunique()
inconsistencias = conteo_paises[conteo_paises > 1]

if not inconsistencias.empty:
    print("Hay inconsistencias entre el pais y su codigo_iso")
    display(df[df['codigo_iso'].isin(inconsistencias.index)][['codigo_iso', 'pais']].drop_duplicates())
else:
    print("No hay inconsistencias entre el pais y su codigo_iso")

No hay inconsistencias entre el pais y su codigo_iso





### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

> Puedes usar la siguiente lista para filtrar el DataFrame final por la columna `codigo_iso`.

In [74]:
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']

df_america = df[df['codigo_iso'].isin(america)].copy()

print("Paises con mayor/menor indice por año:")
for year in df_america['anio'].unique():
    df_year = df_america[df_america['anio'] == year]

    #Manejo de casos en donde puede no haber datos de año
    if not df_year['indice'].isnull().all():
        min_indice_pais = df_year.loc[df_year['indice'].idxmin()]['pais']
        max_indice_pais = df_year.loc[df_year['indice'].idxmax()]['pais']

        print(f"Año {year}:")
        print(f"  Pais con menor indice (mayor libertad): {min_indice_pais}")
        print(f"  Pais con mayor indice (menor libertad): {max_indice_pais}")
    else:
        print(f"Año {year}: No hay datos disponibles.")

Paises con mayor/menor indice por año:
Año 2001:
  Pais con menor indice (mayor libertad): Canadá
  Pais con mayor indice (menor libertad): Cuba
Año 2002:
  Pais con menor indice (mayor libertad): Trinidad y Tobago
  Pais con mayor indice (menor libertad): Cuba
Año 2003:
  Pais con menor indice (mayor libertad): Trinidad y Tobago
  Pais con mayor indice (menor libertad): Argentina
Año 2004:
  Pais con menor indice (mayor libertad): Trinidad y Tobago
  Pais con mayor indice (menor libertad): Cuba
Año 2005:
  Pais con menor indice (mayor libertad): Bolivia
  Pais con mayor indice (menor libertad): Cuba
Año 2006:
  Pais con menor indice (mayor libertad): Canadá
  Pais con mayor indice (menor libertad): Cuba
Año 2007:
  Pais con menor indice (mayor libertad): Canadá
  Pais con mayor indice (menor libertad): Cuba
Año 2008:
  Pais con menor indice (mayor libertad): Canadá
  Pais con mayor indice (menor libertad): Cuba
Año 2009:
  Pais con menor indice (mayor libertad): Estados Unidos
  Pais 

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.

In [78]:
# Filtrar por países de América y eliminar filas con 'indice' faltante
df_america_limpio = df[df['codigo_iso'].isin(america)].dropna(subset=['indice']).copy()

print("Países con índice mínimo/máximo por año (usando groupby):")

# Encontrar el país con el índice mínimo por año
min_indice_por_anio = df_america_limpio.loc[df_america_limpio.groupby('anio')['indice'].idxmin()]
print("\nPaís con índice mínimo por año:")
display(min_indice_por_anio[['anio', 'pais', 'indice']])

# Encontrar el país con el índice máximo por año
max_indice_por_anio = df_america_limpio.loc[df_america_limpio.groupby('anio')['indice'].idxmax()]
print("\nPaís con índice máximo por año:")
display(max_indice_por_anio[['anio', 'pais', 'indice']])

Países con índice mínimo/máximo por año (usando groupby):

País con índice mínimo por año:


,anio,pais,indice
27,2001,Canadá,0.80
343,2002,Trinidad y Tobago,1.00
523,2003,Trinidad y Tobago,2.00
703,2004,Trinidad y Tobago,2.00
741,2005,Bolivia,4.50
927,2006,Canadá,4.88
1107,2007,Canadá,3.33
1287,2008,Canadá,3.70
1611,2009,Estados Unidos,6.75
1701,2012,Jamaica,9.88



País con índice máximo por año:


,anio,pais,indice
39,2001,Cuba,90.30
219,2002,Cuba,97.83
365,2003,Argentina,35826.00
579,2004,Cuba,87.00
759,2005,Cuba,95.00
939,2006,Cuba,96.17
1119,2007,Cuba,88.33
1299,2008,Cuba,94.00
1479,2009,Cuba,78.00
1659,2012,Cuba,71.64


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.

In [79]:
pivot_table_indice = pd.pivot_table(df, values='indice', index='pais', columns='anio', aggfunc='max', fill_value=0)
display(pivot_table_indice.head())

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Afghanistán,35.5,40.17,28.25,39.17,44.25,56.50,59.25,54.25,51.67,37.36,37.07,37.44,37.75,39.46,37.28,36.55
Albania,0.0,6.50,11.50,14.17,18.00,25.50,16.00,21.75,21.50,30.88,29.92,28.77,29.92,29.92,29.49,29.84
Alemania,1.5,1.33,2.00,4.00,5.50,5.75,4.50,3.50,4.25,10.24,10.23,11.47,14.80,14.97,14.39,14.60
Algeria,31.0,33.00,43.50,40.33,40.00,40.50,31.33,49.56,47.33,36.54,36.26,36.63,41.69,42.83,43.13,45.75
Andorra,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.82,6.82,19.87,19.87,21.03,22.21,24.63


#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?

In [87]:
tabla_pivote_indice = pivot_table_indice
max_indice_total = tabla_pivote_indice.max().max()
pais_max_indice = tabla_pivote_indice[tabla_pivote_indice == max_indice_total].stack().idxmax()[0]

print(f"País con el índice más alto: {pais_max_indice} ({max_indice_total})")

min_indice_total = tabla_pivote_indice[tabla_pivote_indice > 0].min().min()
pais_min_indice = tabla_pivote_indice[tabla_pivote_indice == min_indice_total].stack().idxmin()[0]

print(f"País con el índice no nulo más bajo: {pais_min_indice} ({min_indice_total})")

País con el índice más alto: Kosovo (64536.0)
País con el índice no nulo más bajo: Austria (0.5)


**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

In [86]:
tabla_pivote_indice = pivot_table_indice

promedio_indice_por_anio = tabla_pivote_indice.mean(axis=0)

anio_promedio_mas_alto = promedio_indice_por_anio.idxmax()
promedio_mas_alto = promedio_indice_por_anio.max()

anio_promedio_mas_bajo = promedio_indice_por_anio.idxmin()
promedio_mas_bajo = promedio_indice_por_anio.min()

print(f"Año con el promedio de índice más alto: {anio_promedio_mas_alto} ({promedio_mas_alto:.2f})")
print(f"Año con el promedio de índice más bajo: {anio_promedio_mas_bajo} ({promedio_mas_bajo:.2f})")

Año con el promedio de índice más alto: 2013 (449.11)
Año con el promedio de índice más bajo: 2001 (20.03)


**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

In [85]:
tabla_pivote_indice = pivot_table_indice

variabilidad_indice = tabla_pivote_indice.max(axis=1) - tabla_pivote_indice.min(axis=1)

pais_max_variabilidad = variabilidad_indice.idxmax()
valor_max_variabilidad = variabilidad_indice.max()

print(f"País con mayor variabilidad en el índice: {pais_max_variabilidad} ({valor_max_variabilidad:.2f})")

País con mayor variabilidad en el índice: Kosovo (64536.00)


**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

In [89]:
tabla_pivote_indice = pivot_table_indice

desviacion_std_indice = tabla_pivote_indice.std(axis=1)

paises_indice_constante = desviacion_std_indice[desviacion_std_indice < 1e-9]

if not paises_indice_constante.empty:
    print("Países con índice constante en todos los años:")
    display(paises_indice_constante)
else:
    print("No hay países con índice constante en todos los años.")

No hay países con índice constante en todos los años.


**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?

In [91]:
tabla_pivote_indice = pivot_table_indice

paises_todo_ceros = tabla_pivote_indice[tabla_pivote_indice.sum(axis=1) == 0]

if not paises_todo_ceros.empty:
    print("Países con todos los valores en cero en la tabla pivote:")
    display(paises_todo_ceros)
else:
    print("No hay países con todos los valores en cero en la tabla pivote.")

No hay países con todos los valores en cero en la tabla pivote.


No hay paises con todos los valores en cero, sin embargo si hubiesen habido, esos países probablemente tendrían todos los valores en cero pues no tendrían datos (valores NaN) para la columna 'indice' en el DataFrame original (df) para los años incluidos en la tabla pivote. Como la tabla pivote fue creada con `fill_value=0`, los valores faltantes fueron reemplazados por 0.